# 1. Import dependencies

In [2]:
import os
import re
import pandas as pd
import glob
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer
from typing import List

# 2. Import LLM API key

In [5]:
load_dotenv()
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

if not DASHSCOPE_API_KEY:
    raise ValueError("请在 .env 文件中设置 DASHSCOPE_API_KEY")

# 3. Loading data

In [3]:
file_path = "hk01_scam_articles.md"
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 按 "---" 分割文章（支持前后有空行）
articles = [a.strip() for a in re.split(r'\n-{3,}\n', content) if a.strip()]

print(f"✅ 共加载 {len(articles)} 篇诈骗新闻")

✅ 共加载 606 篇诈骗新闻


# 4. Converting data to document format makes it easier to store in the Chroma database.

In [4]:
docs = []
for i, art in enumerate(articles):
    lines = art.splitlines()
    # 提取标题：取第一行，去掉可能的 Markdown 标题符号（如 ###、##、#）
    if lines:
        title_line = lines[0].strip()
        # 移除开头的 # 符号和空格
        title = re.sub(r'^#+\s*', '', title_line)
    else:
        title = f"文章 #{i+1}"
    
    doc = Document(
        page_content=art,
        metadata={
            "source": file_path,
            "article_id": i + 1,
            "title": title
            # 注意：没有 date 字段
        }
    )
    docs.append(doc)

print("📄 示例标题:", docs[1].metadata["title"] if docs else "无")

📄 示例标题: [2. 網上情緣騙案｜扮女人氹落疊稱贈遺產　冒警二次詐騙　共呃4400萬](https://www.hk01.com/突發/60318574/網上情緣騙案-扮女人氹落疊稱贈遺產-冒警二次詐騙-共呃4400萬)


# 5. Build a vector database

In [6]:
class LocalEmbeddings:
    def __init__(self, model_name="shibing624/text2vec-base-chinese"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, normalize_embeddings=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.model.encode([text], normalize_embeddings=True)[0].tolist()

In [7]:
embeddings = LocalEmbeddings()
# 构建向量库（使用你已定义的 LocalEmbeddings）
vectorstore = Chroma.from_documents(
    docs,
    embeddings,  # ← 你之前的 LocalEmbeddings 实例
    persist_directory="./chroma_hk01_scam_db"  # 建议新建目录，避免混淆
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print(f"写入后文档总数: {vectorstore._collection.count()}")

Loading weights: 100%|█████████████████████| 199/199 [00:00<00:00, 654.61it/s, Materializing param=pooler.dense.weight]
BertModel LOAD REPORT from: shibing624/text2vec-base-chinese
Key                          | Status     |  | 
-----------------------------+------------+--+-
bert.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


写入后文档总数: 606


# 6. Configure LLM and prompt

In [8]:
llm = ChatOpenAI(
    model="deepseek-v3.2",
    openai_api_key=DASHSCOPE_API_KEY,
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0.0
)

In [9]:
prompt_template = """
你是一名反诈专家。请根据以下短信内容和近期真实诈骗新闻案例，判断该短信是否属于已知诈骗模式。

短信内容：
{sms}

相关近期诈骗新闻（来自 HK01）：
{retrieved_cases}

请重点分析：
- 是否涉及相同话术（如“验证身份”、“紧急转账”、“中奖”等）
- 是否模仿官方机构（银行、公安、快递）
- 是否诱导点击链接/下载APP/提供验证码

请严格按照以下 JSON 格式输出（不要任何其他文字）：
{{"score": 整数（0-10）, "reason": "简要理由（50字内）"}}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

# 7. Configure RAG + LLM output

In [10]:
def predict_fraud_probability(sms_text: str) -> tuple[float, str]:
    retrieved_docs = retriever.invoke(sms_text)
    cases = "\n".join([doc.page_content for doc in retrieved_docs])
    
    formatted_prompt = prompt.format(sms=sms_text, retrieved_cases=cases)
    response = llm.invoke(formatted_prompt)
    output = response.content.strip()
    
    try:
        # 尝试解析 JSON
        result = json.loads(output)
        score = int(result["score"])
        reason = result.get("reason", "无理由").strip()
        score = max(0, min(10, score))
        probability = score / 10.0
        return probability, reason
    except Exception as e:
        # fallback：从文本中提取数字和理由
        numbers = re.findall(r'\d+', output)
        score = int(numbers[0]) if numbers else 5
        score = max(0, min(10, score))
        probability = score / 10.0
        
        # 提取理由（找冒号/引号后的内容）
        reason_match = re.search(r'(?<=[：:"])([^"]+?)(?="|$)', output)
        if reason_match:
            reason = reason_match.group(1).strip()[:60]
        else:
            reason = f"（LLM 返回：{output[:30]}...）"
        
        return probability, reason

# 8. Configure LLM output

In [13]:
def predict_fraud_probability_baseline(sms_text: str) -> tuple[float, str]:
    """
    纯 DeepSeek-v3.2 判断（无 RAG），使用与 RAG 版本完全相同的 Prompt 和解析逻辑
    """
    # ⚠️ 注意：这里不再调用 retriever，retrieved_cases 固定为 "无"
    formatted_prompt = prompt.format(sms=sms_text, retrieved_cases="无")
    response = llm.invoke(formatted_prompt)
    output = response.content.strip()

    try:
        # 尝试解析 JSON
        result = json.loads(output)
        score = int(result["score"])
        reason = result.get("reason", "无理由").strip()
        score = max(0, min(10, score))
        probability = score / 10.0
        return probability, reason
    except Exception as e:
        # fallback：从文本中提取数字和理由
        numbers = re.findall(r'\d+', output)
        score = int(numbers[0]) if numbers else 5
        score = max(0, min(10, score))
        probability = score / 10.0

        # 提取理由（找冒号/引号后的内容）
        reason_match = re.search(r'(?<=[：:"])([^"]+?)(?="|$)', output)
        if reason_match:
            reason = reason_match.group(1).strip()[:60]
        else:
            reason = f"（LLM 返回：{output[:30]}...）"

        return probability, reason

# 9. Test samples

In [17]:
test_sms = """場景：你一個人坐港鐵，車廂唔算太擠。突然兩個打扮時尚、可愛（化妝精緻、穿得青春）的女生（約20-25歲）行過嚟，其中一個輕輕掂咗你膊頭，笑住開口。
女生A（笑容甜美、語氣興奮）：喂～靚仔／小姐～你係唔係香港人呀？（邊講邊笑，望住你等回應）
你：（可能點頭或答「係」）
女生A：哈哈太好了！其實我哋同朋友玩緊一個game，輸咗呀～罰則係要搵3個陌生人抄牌（抄WhatsApp／電話／IG），證明完成任務先可以過關～你幫下手啦，好唔好？好快架～
女生B（另一個女生即刻拎出手機或紙筆，笑住補充）：係呀係呀，就抄個WhatsApp號碼或者IG ID就得～我哋唔會亂send野俾你，純粹證明搵到人幫手～你睇，我哋已經抄咗兩個啦，只剩一個！（展示手機上已經有兩個假的聯絡人列表，增加可信度）
你：（如果猶豫）咁……抄牌做咩？
女生A（即刻轉可愛撒嬌模式）：哎呀就係遊戲罰則咋～輸咗要請食飯添！好尷尬呀～你幫下啦，唔會有事架～（邊講邊靠近少少，製造「親切」氣氛）
女生B（如果見你仲猶豫，就加碼）：或者你畀個IG我加你都OK～之後我delete架～保證唔煩你！快啲啦，列車到站就嚟唔切～"""
prob, reason = predict_fraud_probability(test_sms)
prob_raw, reason_raw = predict_fraud_probability_baseline(test_sms)

print(f"短信内容：{test_sms}")
print("RAG + LLM")
print(f"诈骗概率：{prob:.1%}")
print(f"理由：{reason}")
print("RAW LLM")
print(f"诈骗概率：{prob_raw:.1%}")
print(f"理由：{reason_raw}")

短信内容：場景：你一個人坐港鐵，車廂唔算太擠。突然兩個打扮時尚、可愛（化妝精緻、穿得青春）的女生（約20-25歲）行過嚟，其中一個輕輕掂咗你膊頭，笑住開口。
女生A（笑容甜美、語氣興奮）：喂～靚仔／小姐～你係唔係香港人呀？（邊講邊笑，望住你等回應）
你：（可能點頭或答「係」）
女生A：哈哈太好了！其實我哋同朋友玩緊一個game，輸咗呀～罰則係要搵3個陌生人抄牌（抄WhatsApp／電話／IG），證明完成任務先可以過關～你幫下手啦，好唔好？好快架～
女生B（另一個女生即刻拎出手機或紙筆，笑住補充）：係呀係呀，就抄個WhatsApp號碼或者IG ID就得～我哋唔會亂send野俾你，純粹證明搵到人幫手～你睇，我哋已經抄咗兩個啦，只剩一個！（展示手機上已經有兩個假的聯絡人列表，增加可信度）
你：（如果猶豫）咁……抄牌做咩？
女生A（即刻轉可愛撒嬌模式）：哎呀就係遊戲罰則咋～輸咗要請食飯添！好尷尬呀～你幫下啦，唔會有事架～（邊講邊靠近少少，製造「親切」氣氛）
女生B（如果見你仲猶豫，就加碼）：或者你畀個IG我加你都OK～之後我delete架～保證唔煩你！快啲啦，列車到站就嚟唔切～
RAG + LLM
诈骗概率：80.0%
理由：符合街头搭讪收集个人信息模式，虽未直接索财，但以游戏为名骗取联系方式，属诈骗前奏。
RAW LLM
诈骗概率：30.0%
理由：模式类似街头搭讪收集个人信息，虽与典型电诈骗话术不同，但存在后续滋扰或诈骗风险。
